In [1]:
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon

# Load Data and Filter down to Study area parcels

In [2]:
bk_prices = pd.read_csv('data/brooklyn_prices.csv', index_col=0)

In [3]:
pluto = gpd.read_file('data/pluto.geojson')

DriverError: 'data/pluto.geojson' not recognized as a supported file format.

In [ ]:
addr = "373 VAN BRUNT STREET"
addr_pluto = pluto[pluto['address'] == addr]

addr_pluto_geometry = addr_pluto['geometry']

addr_coors = [float(addr_pluto_geometry.centroid.y), float(addr_pluto_geometry.centroid.x)]
addr_point =  Point(addr_coors)
print(type(addr_coors[0]))

In [ ]:
# get study area pluto data
radius = 0.3 # miles
study_area = addr_point.buffer(radius / 69)
study_area_polygon = Polygon(study_area.exterior.coords)

def centroid_to_coor(centroid):
    return Point(centroid.y, centroid.x)

study_area_pluto = pluto[pluto.apply(lambda x: study_area_polygon.contains(centroid_to_coor(x['geometry'].centroid)), axis=1)]
len(study_area_pluto)

# Join Rolling Data and Pluto

In [ ]:
def format_bbl(parcel):
    borough = str(parcel['BOROUGH'])
    block = str(parcel['BLOCK'])
    lot = str(parcel['LOT'])
    return int(borough + block.zfill(5) + lot.zfill(4))

In [ ]:
# Create bbls
bk_prices['BBL'] = bk_prices.apply(format_bbl, axis=1)


In [ ]:
study_area_prices = bk_prices[bk_prices.apply(lambda parcel: parcel['BBL'] in study_area_pluto['bbl'].values, axis=1)]

In [ ]:
sold_parcels = study_area_pluto[study_area_pluto.apply(lambda parcel: parcel['bbl'] in study_area_prices['BBL'].values, axis=1)]

# Analyze the study area sales

In [ ]:
import matplotlib.pyplot as plt

# how many parcels were sold multiple times
# what are the transactions

In [ ]:
list(study_area_prices)

# Visualize

In [ ]:
import folium
import geopandas
m = folium.Map(location=addr_coors,zoom_start=14,tiles="Stamen Toner")

folium.Circle(addr_coors, radius=radius * 1609.34).add_to(m)

folium.GeoJson(
    sold_parcels['geometry'],
    name='geojson'
).add_to(m)

In [ ]:
def style_function(feature):
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black'
    }

In [ ]:
folium.GeoJson(
    addr_pluto_geometry,
    name='geojson',
    style_function=style_function
).add_to(m)

In [ ]:
m

In [ ]:
print("{} percent are 0$".format(len(study_area_prices[study_area_prices['SALE PRICE'] == 0]) / len(study_area_prices) * 100))

In [ ]:
print(len(study_area_prices))
non_zero_sales = study_area_prices[study_area_prices['SALE PRICE'] != 0]
print(len(non_zero_sales))
non_zero_sales

In [ ]:
fig = plt.figure(figsize=(17, 6))
plt.scatter(non_zero_sales['SALE DATE'], non_zero_sales['SALE PRICE'] / non_zero_sales['GROSS SQUARE FEET'], s=5)
plt.xticks(rotation=90)
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt